In [1]:
# from: 20211221+/distilbert_load_eval.ipynb

# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import os
import pandas as pd
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
import json
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_goscv_solution_distilbert_1'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def read_passages(path_data):
    df = pd.read_csv(path_data)
    documents = df['processed_log'].to_list()
    return documents
# end

In [5]:
# call the function
# path_folder_data = 'datasource'
# name_data = 'train-goscv-solution-202206211830.csv'
# path_data = os.path.join(path_folder_data, name_data)
# valid_texts = read_passages(path_data)

In [6]:
valid_texts = ["timestamp failed at play deploy vm efi nvme vmxnet number timestamp task upload local file to esxi data store task path home worker workspace dw rhel number arm ansible vsphere gos validation common esxi upload data store file yml number exception in vsphere copy python when main in request python when http error default fatal localhost failed http error number not found timestamp task testing exit due to failure task path home worker workspace dw rhel number arm ansible vsphere gos validation common test rescue yml number fatal localhost failed exit testing when exit testing when fail is set to true in test case deploy vm efi nvme vmxnet number"] * 64

In [7]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if len(os.listdir(model_dir)) > 1:
    info_model = model_dir
    print('redirect model to use local files {}'.format(info_model))
else:
    info_model = model_name
    print('redirect model to use official {}'.format(info_model))
# end

redirect model to use local files model_goscv_solution_distilbert_1


In [8]:
filename_label = 'labels.json'
path_label = os.path.join(info_model, filename_label)
with open(path_label, 'r') as file:
    labels = json.load(file)
# end
target_names = labels
print(target_names)

['deepdive', 'retry']


In [9]:

model = DistilBertForSequenceClassification.from_pretrained(info_model, num_labels=len(target_names))
model = model.cuda()

In [18]:
list_conf = []
list_label = []

for text_valid in valid_texts:
    input_valid_tokenized = tokenizer(text_valid, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to('cuda')
    with torch.no_grad():
        out = model(**input_valid_tokenized, output_hidden_states=True, output_attentions=True)
    # end

    probas_evaluate = torch.nn.functional.softmax(out.logits, dim=-1)
    answer_evaluate = int(probas_evaluate.argmax().cpu())
    
    label_evaluate = target_names[answer_evaluate]
#     list_conf.append(max(probas_evaluate.cpu().numpy().tolist()[0]))
    list_conf.append(probas_evaluate.cpu().numpy().tolist()[0])
    list_label.append(label_evaluate)
# end

In [22]:
list_conf

[[0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189420215785503, 0.9918105602264404],
 [0.008189

In [11]:
# path_folder_data = 'data'
# name_data_origin = 'log_content_5_20220507_target.csv'
# name_data_target = 'log_content_5_20220507_target_priority.csv'

# path_data_origin = os.path.join(path_folder_data, name_data_origin)
# path_data_target = os.path.join(path_folder_data, name_data_target)

# data_all = pd.read_csv(path_data_origin)
# data_all['conf_priority'] = list_conf
# data_all['label_priority'] = list_label

# data_all.to_csv(path_data_target)

In [12]:
# Works for investigation
# text_valid = valid_texts[0]
# input_valid_tokenized = tokenizer(text_valid, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to('cuda')
# with torch.no_grad():
#     out = model(**input_valid_tokenized, output_hidden_states=True, output_attentions=True)
# # end

# probas_main = torch.nn.functional.softmax(out.logits, dim=-1)
# answer_main = int(probas_main.argmax().cpu())

# max(probas_main.cpu().numpy().tolist()[0])